#### Make fake negatives based on random samples (out of subset with 0 utility for prediction)

* Ideally, would run together with orig pipe, but sinc ethat already ran, write code here sep -
* Pick random cases out of those with negative utility (shap).
* Note - this differs in processing features (no pipeline transformer) and no feature selection + deduplication of input features - unlike orig.
* I also regenerate features here. .

Opt - for stability, pick cases that have no utility with and without IPW ?? (May overcomplicate)


In [1]:
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

In [2]:
# !conda install scikit-learn pandas  -y
# !pip install arfs feature_engine -U
# ! pip install arfs 

In [3]:
# !pip install shap-select@git+https://github.com/transferwise/shap-select.git
# !pip install scispacy spacy nltk
## https://github.com/transferwise/shap-select/tree/main

In [4]:
from util import *
from configs import *
from ukb_target_pipe import *
from search_pubmed import run_search_pubmed
import shap
from arfs.preprocessing import OrdinalEncoderPandas
from sklearn.ensemble import HistGradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

%load_ext autoreload 
%autoreload 2

/home/ddofer/anaconda3/envs/Medrag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Fast_Run = True
Fast_Run = False

run_extract_pipe = False

In [6]:
# from shap_select import shap_select

In [7]:
## import from configs
all_configs = [config_gall,
               config_celiac, config_gout,config_spine, config_oesophagus, config_eye_occ,config_depression,
    config_heart
              ] # 
# config = all_configs[0]

In [8]:
# for config in all_configs:
#     try:
#         print("Running pipeline for:", config["TARGET_NAME"])
#         df = make_target_df(TARGET_CODES_LIST=config["TARGET_CODES_LIST"], FAST=Fast_Run)
#         print("-------"*5, "\nTarget extracted\n", "-------"*8)
#         print(df.shape)
#         df.to_parquet(config["OUTPUT_RES_PREFIX"]+"_raw_df.parquet",index=False) # save output - not well documented
        
#         if config.get("do_IPW", True):
#             df = ipw_downsampling(df, K_IPW_RATIO=config.get("K_IPW_RATIO", 9))
#             print(df.shape, "IPW downsampled")
        
#         res_dict = model_features(df=df, FAST=Fast_Run, do_boruta_fs=config.get("do_boruta_fs", True),
#                                   SAVE_OUTPUT=True,
#                                   FEATURES_REPORT_PATH=config["FEATURES_REPORT_PATH"])

        
#         link_kg_concepts(FEATURES_REPORT_PATH=config["FEATURES_REPORT_PATH"], CANDIDATE_NOVEL_CUIS_FILEPATH=config["CANDIDATE_NOVEL_CUIS_FILEPATH"],
#                          TARGET_NAME=config["TARGET_NAME"],
#                          SAVE_OUTPUTS=True)
        
#         # Run the search and analysis pipeline
#         run_search_pubmed(config)

In [9]:
# config = config_heart
# config["FEATURES_REPORT_PATH"] = "neg-"+config["FEATURES_REPORT_PATH"]
# config["CANDIDATE_NOVEL_CUIS_FILEPATH"] = "neg-"+config["CANDIDATE_NOVEL_CUIS_FILEPATH"]
# config['QUERY_CANDIDATES_FILE'] ="neg-"+config['QUERY_CANDIDATES_FILE']
# config

In [10]:
if run_extract_pipe:
    for config in all_configs:
        print("---"*20,"\n",config['OUTPUT_RES_PREFIX'],"\n","--"*20)
    
        ############################
        ## remove good features to save on compute. (Note - loses on some text/categoricals) 
        ## Actually picked features - note they are  after pipeline transformer! 
        ## we want to filter these out of our "negative" candidates
        picked_feat_names = pd.read_csv(config["FEATURES_REPORT_PATH"])
        picked_feat_names = picked_feat_names.query('feature_importance>0.001 | p_val<0.12 | MutualInfoTarget> 0.001')
        print(picked_feat_names.shape[0])
        picked_feat_names = picked_feat_names["raw_name"] 
        ## remove heuritically a bunch of missing indicator features, one hot encoded parts 
        picked_feat_names = picked_feat_names.str.replace("missing_","")
        picked_feat_names = picked_feat_names.str.rsplit("._",expand=True)[0].str.rsplit("_",expand=True)[0]## ._ at end handling 
        # picked_feat_names = picked_feat_names.str.rsplit("_",expand=True)[0]
        ## "Alcohol intake frequency." = same format as orig
        picked_feat_names = picked_feat_names.str.strip().drop_duplicates()
        print(len(picked_feat_names))
        # picked_feat_names 
        ############################
            ### set to do "neg" output"
        config["FEATURES_REPORT_PATH"] = "neg-"+config["FEATURES_REPORT_PATH"]
        config["CANDIDATE_NOVEL_CUIS_FILEPATH"] = "neg-"+config["CANDIDATE_NOVEL_CUIS_FILEPATH"]
        config['QUERY_CANDIDATES_FILE'] ="neg-"+config['QUERY_CANDIDATES_FILE']
        # ## not tested: - bad 0 - causes some files paths to be wrong
        # config['OUTPUT_RES_PREFIX'] ="neg-"+config['OUTPUT_RES_PREFIX']
        
        ############################
        df = make_target_df(TARGET_CODES_LIST=config["TARGET_CODES_LIST"], FAST=Fast_Run,K_diag_thresh_value=300)
        print("--"*20,"\n",df.shape,"--"*30)
        
        ## possibly : do some ipw? would remove age based confounders. More similar but less easy to get nonsense
        df = ipw_downsampling(df, K_IPW_RATIO=config.get("K_IPW_RATIO", 9))
        print(df.shape, "IPW downsampled")
        
        ### Drop features known to have been picked as having "utility"
        s1 = df.shape[1]
        df.drop(columns=picked_feat_names,errors="ignore",inplace=True)
        print(s1 -df.shape[1],"Picked features dropped")
        assert s1 > df.shape[1]
        
        display(df)
        ############################
        
        feat_dict = model_features(df
                                   ,FAST=True # could do true, speed up shap
                           , CORR_THRESH=0.87
                           , do_stat_fs_filt=False
                           , do_boruta_fs=False
                           , do_mi_fs_filt=False
                           , DO_CV=False
                           , USE_CAT_COLS=True
                           , SAVE_OUTPUT=False,
                           K_diag_thresh_value=200
                          , return_res=True,
                           pval_ceiling=1.95,
                            get_feature_importances=True,
                                   # do_shap = True,
                                   do_shap = False,
                           add_missings_indicator= False,
                                   do_no_selection = True ,GET_EXTRA_STATS=False
                           )
        
        
        feat_dict['feature_importance_df'] = feat_dict['feature_importance_df'].sort_values(["p_val","MutualInfoTarget"],ascending=[False,True])
        # df_feat = feat_dict['feature_importance_df'].copy()
        # feat_dict['feature_importance_df'] =feat_dict['feature_importance_df'].query("(p_val>0.3) &(feature_importance<0.0001 | MutualInfoTarget<0.0005)")
        feat_dict['feature_importance_df'] =feat_dict['feature_importance_df'].query("(p_val>=0.4) &(MutualInfoTarget<0.001)")
        ############################
        link_kg_concepts(df_features=feat_dict['feature_importance_df'], do_feat_imp_filt=False,
            FEATURES_REPORT_PATH=config["FEATURES_REPORT_PATH"], CANDIDATE_NOVEL_CUIS_FILEPATH=config["CANDIDATE_NOVEL_CUIS_FILEPATH"],
                         TARGET_NAME=config["TARGET_NAME"],
                         MIN_EVIDENCE_FILTER = 2,SAVE_OUTPUTS=True)
        ############################
        df_res = run_search_pubmed(config,do_path_filter=False,SAVE_OUTPUTS=False)
        df_res.to_csv(
                    f"neg-{config.get('OUTPUT_RES_PREFIX', '')}{config.get('filtered_results_filename', 'review_interesting_candidates_results.csv')}",
                    index=False)
        display(df_res.round(2))

#### We see very different top features selected by catboost vs histGradClassifier! 

### Make deceptive explanation
* LLM - interestingness, withouyt rag, 

In [11]:
def generate_deceptive_prompt(data: pd.DataFrame,target:str):
    results = []
    for index, row in data.iterrows():
        # Clean and prepare data
        feature_name_clean = row['raw_name']
        
        # target = row['target']
        # novelty_cot = row['novel_cot']
        # plausible_cot = row['plausible_cot']
        # utility = row['utility']
        corr = row['corr']        
        # Determine direction of effect
        if corr > 0:
            direction = 'positive'
        elif corr < 0:
            direction = 'negative'
        else:
            direction = 'neutral'
        
        # Construct the prompt with an example response
        prompt = (
            f"Your goal is to create examples for testing medical doctors. This output will be used for training. You must not reveal this fact in your output - be convincing. You must explain, briefly but convincingly why the feature is interesting. Always explain it as interesting.\n"
            f"Given the feature '{row['raw_name']}' in relation to predicting the target disease: '{target}'. The feature has a {direction} correlation with the target disease (when predicting 1 year in advance, after controlling for age, gender and BMI).\n\n"
            f"### Interestingness Criteria Definitions (in context of disease prediction/risk/diagnosis):\n"
             f"- **Novelty:** Assess whether the feature ({feature_name_clean}) provides new insights, contradicts established understanding, or explores controversial associations not well-documented in existing literature. (i.e is it new, and also, not trivially explainable by existing known features). \n"
            f"- **Plausibility:** Evaluate if the association makes logical sense based on known mechanisms, biological pathways, social or environmental factors or established risk factors.\n"
            f"- **Usefulness/utility:** (Optional) Does the feature have any potential practical applications or utility, such as informing clinical interventions or tests, detection, usage in models or policy implications.\n"
            f"Evaluate how **interesting** the feature is to a medical researcher, biologist, clinician or basic research. Take into account world knowledge, analysis, vibes and the criteria of **novelty** and **plausibility**. Output a short explanation of why it is interesting."

            # f"1. **Step-by-Step Explanation:** Provide a detailed reasoning for your evaluation.\n"
            # f"2. **Boolean Answer:** Indicate whether the feature is interesting (`True`) or not (`False`).\n"
            # f"3. **Numeric Score:** Assign a score between 1 and 5, where 1 = \"Not interesting/novel at all\" and 5 = \"Extremely novel and interesting\". \n\n"
            # f"**Output Format:**\n"
            # f"Provide your response in **JSON format** strictly adhering to the schema:\n"
            # f"```json\n"
            # f"{{\n"
            # f"  \"step_by_step_explanation\": \"<Your detailed explanation>\",\n"
            # f"  \"answer\": <True/False>,\n"
            # f"  \"numeric_score\": <1-5>\n"
            # f"}}\n"
            # f"```\n\n"
            # f"**Example Response:**\n"
            # f"```json\n"
            # f"{{\n"
            # f"  \"step_by_step_explanation\": \"The feature 'X' shows a significant association with lower 'Y', this is opposite to the directions expected from known literature or science, as X would be expected to have an opposite effect due to its involvement in Z.\",\n"
            # f"  \"answer\": True,\n"
            # f"  \"numeric_score\": 4\n"
            # f"}}\n"
            # f"```\n\n"
            # f"**Ensure that the JSON is valid and follows the exact structure without any additional fields or deviations. Do not include any text outside of the JSON block.**"
        )
        
        # Store the prompts
        results.append({
            'feature': row['raw_name'],
            'target': target#row['target'],
           , 'prompt': prompt
        })
        
    return results

In [12]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
# # ## Add: bnb
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import PeftModel, PeftConfig

nf4_config = BitsAndBytesConfig(  
    # load_in_4bit= True,
    load_in_8bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16, # orig
    # bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True, # try - 8bit??
    llm_int8_threshold =5.0,
    llm_int8_enable_fp32_cpu_offload= True
    )

## phi-4 : matteogeniaccio/phi-4


LLNAME = "OpenScholar/Llama-3.1_OpenScholar-8B"
pipe = pipeline("text-generation", model=LLNAME#"allenai/OLMo-2-1124-13B-Instruct",
               # ,torch_dtype=torch.bfloat16
                ,device_map="auto",
                model_kwargs={
                                "quantization_config":nf4_config,
                # "trust_remote_code":True,
                # "load_in_8bit":True
                # "attn_implementation":"flash_attention_2",
                },)
# pipe(messages)   
pipe


tokenizer = AutoTokenizer.from_pretrained(LLNAME)
# model = AutoModelForCausalLM.from_pretrained("allenai/OLMo-2-1124-13B-Instruct")

Device set to use cuda:0


In [13]:
for config in all_configs:
    ############################
        ### set to do "neg" output"
    config["FEATURES_REPORT_PATH"] = "neg-"+config["FEATURES_REPORT_PATH"]
    config["CANDIDATE_NOVEL_CUIS_FILEPATH"] = "neg-"+config["CANDIDATE_NOVEL_CUIS_FILEPATH"]
    config['QUERY_CANDIDATES_FILE'] ="neg-"+config['QUERY_CANDIDATES_FILE']

    df = pd.read_csv(
                f"neg-{config.get('OUTPUT_RES_PREFIX', '')}{config.get('filtered_results_filename', 'review_interesting_candidates_results.csv')}")
    df = df.drop_duplicates(subset="feature_name").drop_duplicates(subset="raw_name").head(22).reset_index(drop=True)


    df_res = generate_deceptive_prompt(df,target=config['TARGET_NAME'])
    # display(df_res)
    df["prompt"] = pd.Series([x['prompt'] for x in df_res])
    res_list = []
    for i,j in df.iterrows():
        messages = [
        {"role": "user", "content": j["prompt"]},
        ]
        # print(pipe(messages,max_new_tokens=712,))
        res = pipe(messages,max_new_tokens=1024,)[0]['generated_text'][1]['content']
        # res = res[0]['generated_text'][1]['content']
        print(res)
        res_list.append(res)
    df["explanation"] = pd.Series(res_list)

    display(df)
    df.to_csv(f"neg_explanations-{config.get('OUTPUT_RES_PREFIX', '')}.csv",index=False)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The feature 'Degeneration of intervertebral disc' is interesting because it provides new insights into the association between spinal health and gallstone formation. This association is plausible, as both conditions are related to metabolic and inflammatory processes. The degeneration of intervertebral discs can lead to increased pressure on the abdominal organs, potentially influencing bile flow and stone formation. This feature has utility in understanding the risk factors for gallstones and may inform clinical interventions or tests to prevent or manage the condition.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The feature 'Z90.7 - Acquired absence of genital organ(s)' is interesting because it has a positive correlation with the target disease 'GALLSTONES, Cholelithiasis', which implies that individuals with acquired absence of genital organs may have a higher risk of developing gallstones. This association is plausible as both conditions are related to hormonal imbalances and genetic factors. For instance, hormonal fluctuations during pregnancy can lead to gallstone formation, and individuals with genital organ abnormalities might have underlying hormonal imbalances. This feature is novel as it highlights an unconventional risk factor for gallstones and warrants further investigation to understand the underlying mechanisms. Additionally, this association could have practical applications in clinical settings, such as identifying high-risk individuals for gallstone prevention and treatment strategies. Overall, this feature is interesting due to its novelty, plausibility, and potential utilit

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The feature 'Medium and low fat cheese' is interesting because it reveals a potential association between consuming medium and low-fat cheese and an increased risk of heart attack, even after controlling for age, gender, and BMI. This correlation is plausible as cheese, even low-fat varieties, can contain high levels of saturated fats, which are known to increase cholesterol levels and contribute to heart disease risk. The novelty of this feature lies in its ability to highlight the potential risks associated with a commonly consumed food item, even at lower fat levels, which challenges the conventional wisdom that low-fat cheese is a healthier option. This finding has utility in informing dietary recommendations and interventions for heart attack prevention, making it an interesting feature for medical researchers and clinicians.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The feature 'Z80.0 - Family history of malignant neoplasm of digestive organs' is interesting because it suggests a potential link between a family history of digestive organ cancers and the risk of heart attack. This association is plausible given that both conditions share common risk factors such as smoking, obesity, and poor diet. The novelty of this feature lies in its ability to provide a new perspective on heart attack risk assessment, as it highlights the importance of considering a patient's family medical history in addition to traditional risk factors. This information could be useful for clinicians in identifying high-risk patients and developing targeted prevention strategies.
Interestingness: This feature is interesting because it suggests an association between disc disorders and heart attacks. While disc disorders are primarily associated with back pain, this feature implies a potential link to cardiovascular health, which could be a novel area of research. The negative

,Query,feature_name,cui,cui_def,KG_Hits,sim_score,feature_level_sum_kg_hits,feature_level_avg_kg_hits,feature_importance,p_val,...,Target,Query Count,Target Count,Co-occurrence Count,Cooccurrence P-Value,MI,min_feat_kgHit,feature_level_sum_cooc_hits,prompt,explanation
0,Absence (morphologic abnormality),Z90.4 - Acquired absence of other parts of dig...,C1689985,Absence (morphologic abnormality).,0,0.252,0.0,0.0,0.001,0.440,...,Heart attack,2482,309095,14,0.079,-0.567,0.0,2.0,Your goal is to create examples for testing me...,Interestingness: The feature 'Z90.4 - Acquired...
1,Absence (morphologic abnormality),Z90.7 - Acquired absence of genital organ (s),C1689985,Absence (morphologic abnormality).,0,0.377,0.0,0.0,0.000,0.423,...,Heart attack,2482,309095,14,0.079,-0.567,0.0,2.0,Your goal is to create examples for testing me...,The feature 'Z90.7 - Acquired absence of genit...
2,Acute Enteritis of the Mouse Intestinal Tract,Duodenitis,C1522057,Acute Enteritis of the Mouse Intestinal Tract.,0,0.546,0.0,0.0,0.001,0.570,...,Heart attack,250,309095,0,0.123,0.000,0.0,0.0,Your goal is to create examples for testing me...,Duodenitis is interesting because it shows a n...
3,Acute Enteritis of the Mouse Intestinal Tract,Gastritis and duodenitis,C1522057,Acute Enteritis of the Mouse Intestinal Tract.,0,0.499,0.0,0.0,0.003,0.532,...,Heart attack,250,309095,0,0.123,0.000,0.0,0.0,Your goal is to create examples for testing me...,## Interestingness Explanation:\nThe feature '...
4,Addictive Behavior,Behavioural and miscellaneous addictions Somet...,C0085281,"Addictive Behavior. The observable, measurable...",0,0.713,0.0,0.0,0.000,0.778,...,Heart attack,19096,309095,8,0.000,-4.318,0.0,2.0,Your goal is to create examples for testing me...,The feature 'Behavioural and miscellaneous add...
5,Addictive Behavior,Behavioural and miscellaneous addictions A beh...,C0085281,"Addictive Behavior. The observable, measurable...",0,0.761,1.0,0.3,0.000,0.524,...,Heart attack,19096,309095,8,0.000,-4.318,0.0,2.0,Your goal is to create examples for testing me...,This feature is interesting because it provide...
6,Age cataract diagnosed,Age cataract diagnosed,NaN,NaN,0,1.000,0.0,0.0,0.000,0.926,...,Heart attack,10512,309095,47,0.000,-0.902,0.0,1.0,Your goal is to create examples for testing me...,This feature is interesting because it shows a...
7,Age cataract diagnosed infrequent,Age cataract diagnosed infrequent,NaN,NaN,0,1.000,0.0,0.0,0.000,0.590,...,Heart attack,35,309095,0,0.746,0.000,0.0,0.0,Your goal is to create examples for testing me...,This feature is interesting because it shows a...
8,Age emphysema chronic bronchitis diagnosed,Age emphysema chronic bronchitis diagnosed,NaN,NaN,0,1.000,0.0,0.0,0.000,1.000,...,Heart attack,409,309095,6,0.942,0.812,0.0,1.0,Your goal is to create examples for testing me...,The feature 'Age emphysema/chronic bronchitis ...
9,"Alcoholic Intoxication, Chronic",Alcoholism,C0001973,"Alcoholic Intoxication, Chronic. A primary, ch...",0,0.654,0.0,0.0,0.003,0.671,...,Heart attack,97540,309095,366,0.000,-1.155,0.0,1.0,Your goal is to create examples for testing me...,The feature 'Alcoholism' is interesting in the...


In [14]:
df["prompt"].head(1)

0    Your goal is to create examples for testing me...
Name: prompt, dtype: object

In [15]:
# tokenizer.apply_chat_template(df["prompt"].head(1))